In [1]:
!git clone https://github.com/Billl-11/NewGenAI.git

Cloning into 'NewGenAI'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 66 (delta 21), reused 26 (delta 5), pack-reused 0
Receiving objects: 100% (66/66), 216.32 KiB | 4.41 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00


In [26]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel

In [27]:
model_bert = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
Tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",do_lower_case=True)

In [28]:
maxlen = 36

In [29]:
datasets_path = "/content/NewGenAI/Datasets/Datasets Preprocessing & Feature engineering/preprocessed dataset.csv"
df = pd.read_csv(datasets_path)
df.head()

,diseases,symptons
0,Fungal infection,"itching, skin rash, nodal skin eruptions, d..."
1,Fungal infection,"skin rash, nodal skin eruptions, dischromic..."
2,Fungal infection,"nodal skin eruptions, itching, dischromic p..."
3,Fungal infection,"dischromic patches, skin rash, itching"
4,Fungal infection,"skin rash, nodal skin eruptions, itching"


In [30]:
# Create a new column to store the tokenizers
df['tokenizers'] = None


# Iterate through the DataFrame rows and create a tokenizer for each row
for index, row in df.iterrows():

    symptoms = row['symptons']

    encoded_input = Tokenizer(symptoms, padding=True, truncation=True, return_tensors='pt', max_length=maxlen)
    tokenizer = encoded_input.input_ids.numpy()
    tokenizer = np.squeeze(tokenizer)

    df.iloc[index].loc['tokenizers'] = tokenizer

In [31]:
df.head()

,diseases,symptons,tokenizers
0,Fungal infection,"itching, skin rash, nodal skin eruptions, d...","[101, 1122, 7520, 117, 2241, 187, 10733, 117, ..."
1,Fungal infection,"skin rash, nodal skin eruptions, dischromic...","[101, 2241, 187, 10733, 117, 6873, 1348, 2241,..."
2,Fungal infection,"nodal skin eruptions, itching, dischromic p...","[101, 6873, 1348, 2241, 19049, 1116, 117, 1122..."
3,Fungal infection,"dischromic patches, skin rash, itching","[101, 6187, 8167, 18882, 1665, 14879, 117, 224..."
4,Fungal infection,"skin rash, nodal skin eruptions, itching","[101, 2241, 187, 10733, 117, 6873, 1348, 2241,..."


In [32]:
# Create a new column to store the padded token

df['padded_tokenizers'] = None

def pad_with_zeros(arr):
    return np.pad(arr, (0, maxlen - len(arr)), mode='constant', constant_values=0)

df['padded_tokenizers'] = df['tokenizers'].apply(pad_with_zeros)

df.head()

,diseases,symptons,tokenizers,padded_tokenizers
0,Fungal infection,"itching, skin rash, nodal skin eruptions, d...","[101, 1122, 7520, 117, 2241, 187, 10733, 117, ...","[101, 1122, 7520, 117, 2241, 187, 10733, 117, ..."
1,Fungal infection,"skin rash, nodal skin eruptions, dischromic...","[101, 2241, 187, 10733, 117, 6873, 1348, 2241,...","[101, 2241, 187, 10733, 117, 6873, 1348, 2241,..."
2,Fungal infection,"nodal skin eruptions, itching, dischromic p...","[101, 6873, 1348, 2241, 19049, 1116, 117, 1122...","[101, 6873, 1348, 2241, 19049, 1116, 117, 1122..."
3,Fungal infection,"dischromic patches, skin rash, itching","[101, 6187, 8167, 18882, 1665, 14879, 117, 224...","[101, 6187, 8167, 18882, 1665, 14879, 117, 224..."
4,Fungal infection,"skin rash, nodal skin eruptions, itching","[101, 2241, 187, 10733, 117, 6873, 1348, 2241,...","[101, 2241, 187, 10733, 117, 6873, 1348, 2241,..."


In [33]:
pad_len = len(df.iloc[0].loc["padded_tokenizers"])

print(f"Padding length: {pad_len}")

Padding length: 36
